# モデル評価（k-fold法） 

データが十分に大きければ（例えば、訓練データで数百万レコード、検証データで数十万レコードの規模）、holdout法によるCV検証スコア（Cross ValidationのTESTデータの評価値）は安定するでしょうが、そうでない場合、holdoutのCV検証スコアがブレるであろうことは想像できます。そこで実質k回のholdoutを実施しそのCV検証スコアの平均値を比較することで、より精緻にベストモデルの選択判断を行おうというものがk-fold法でした。<br><br>以下実装例を紹介します。

In [1]:
# Load and return the breast cancer wisconsin dataset (classification).
# the Breast Cancer dataset is a classic and very easy binary classification dataset.
import pandas as pd
from sklearn.datasets import load_breast_cancer

dataset = load_breast_cancer()
X = pd.DataFrame(dataset.data, columns=dataset.feature_names)
y = pd.DataFrame(dataset.target, columns=['y'])

holdoutの実装はtrain_test_split、kfoldの実装はcross_val_scoreです。<br>まずは比較のためholdoutを実施。

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline

# 比較のためholdoutの準備
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

# パイプラインのセット
pipe_logistic = Pipeline([('scl',StandardScaler()),('est',LogisticRegression(random_state=1))])

# 学習
pipe_logistic.fit(X_train, y_train.as_matrix().ravel())

# 評価
print('CV_Test:%.6f' % accuracy_score(y_test.as_matrix().ravel(), pipe_logistic.predict(X_test)))

CV_Test:0.982456


次に、k=10のk-fold法によって異なる10個のCV検証スコアを取得します。実装は非常にシンプルで以下1行でできます。データ分割の処理を明示的に書く必要がない点に留意ください。渡しているデータはXとyです。

In [3]:
from sklearn.model_selection import cross_val_score
cv_results = cross_val_score(pipe_logistic, X, y.as_matrix().ravel(), cv=10, scoring='accuracy')
print(cv_results)

[ 0.98275862  0.98275862  0.98245614  0.98245614  0.98245614  0.98245614
  0.94736842  1.          1.          0.98214286]


CV検証スコアの基礎統計量は、例えば以下のように取得できます。

In [4]:
print(pd.DataFrame(cv_results).describe())

               0
count  10.000000
mean    0.982485
std     0.014326
min     0.947368
25%     0.982456
50%     0.982456
75%     0.982759
max     1.000000


以上から、CV検証スコアの平均は0.98程度、標準偏差は0.014と小さく、10回の検証における最小評価値は0.947、最大評価値は1.00とわかりました。実務では、CV検証スコアの平均値を評価すれば十分ですが、最終選択モデルについては、最小スコアと最大スコアが平均スコアから著しく乖離していないかの確認はするようにしましょう。もし乖離が大きい場合、データサンプルを増やせないか、学習データに異常なデータが混入していないかなどの確認をしましょう。